<a href="https://colab.research.google.com/github/Yeon1A/KWHackathon/blob/main/ResNet50_disease_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 라이브러리
!pip install tensorflow-addons

from tensorflow.keras.models import Model
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input,GlobalAveragePooling2D, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, AveragePooling2D, Flatten, Dense, ZeroPadding2D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
import time
import math
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import layers

import random

from keras.preprocessing.image import ImageDataGenerator


os.environ['PYTHONHASHSEED']='1'
os.environ['TF_DETERMINISTIC_OPS']='1'
np.random.seed(5148)
random.seed(5148)
tf.random.set_seed(5148)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
#%% GPU 할당

# GPU를 사용할 수 있는지 확인
#physical_devices = tf.config.list_physical_devices('GPU')
#if len(physical_devices) == 0:
#    print("GPU를 찾을 수 없습니다. CPU를 사용합니다.")
#else:
#    tf.config.experimental.set_memory_growth(physical_devices[0], True)
#    print("GPU를 사용합니다.")


#%% GPU 할당

# GPU를 사용할 수 있는지 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=8*1024)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)


In [ ]:
# 구글 Colab에서 Drive 마운트 (파일을 가져오기 위함)
from google.colab import drive    # google drive mount
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ResNet 50 layer 구조 설계,
def conv1_layer(x):
  x=ZeroPadding2D(padding=(3, 3))(x)
  x=Conv2D(64,(7,7),strides=(2,2))(x)
  x=BatchNormalization()(x)
  x=Activation('relu')(x)
  x=ZeroPadding2D(padding=(1,1))(x)
  return x

In [ ]:
def conv2_layer(x):
    x=MaxPooling2D((3, 3), 2)(x)

    shortcut=x

    for i in range(3):
        if (i == 0):
            x=Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut=Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)
            x=BatchNormalization()(x)
            shortcut=BatchNormalization()(shortcut)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

        else:
            x=Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

    return x

In [ ]:
def conv3_layer(x):
    shortcut = x

    for i in range(4):
        if(i == 0):
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x

In [ ]:
def conv4_layer(x):
    shortcut=x

    for i in range(6):
        if(i == 0):
            x=Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut=Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x=BatchNormalization()(x)
            shortcut=BatchNormalization()(shortcut)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

        else:
            x=Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

    return x

In [ ]:
def conv5_layer(x):
    shortcut = x

    for i in range(3):
        if(i == 0):
            x=Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut=Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x=BatchNormalization()(x)
            shortcut=BatchNormalization()(shortcut)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

        else:
            x=Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x=BatchNormalization()(x)
            x=Activation('relu')(x)

            x=Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x=BatchNormalization()(x)

            x=Add()([x, shortcut])
            x=Activation('relu')(x)

            shortcut=x

    return x

In [ ]:
# Resnet 50layer모델 전체 설계
def resnet50(input_shape,num_classes):
  inputs=Input(shape=input_shape)
  x = conv1_layer(inputs)
  x = conv2_layer(x)
  x = conv3_layer(x)
  x = conv4_layer(x)
  x = conv5_layer(x)
  x = GlobalAveragePooling2D()(x)
  x = Dense(num_classes, activation='softmax')(x)
  model=Model(inputs=inputs,outputs=x)
  return model

In [ ]:
#%% Parameter

input_shape=(224,224,3) #이미지의 크기
num_classes=2 #클래스의 숫자
#disease=["a","b","c","d","e"]

## resnet50 모델 5개를 각각 5개의질병에 적용 (a,b,c,d,e)

model_a=resnet50(input_shape,num_classes)


#%% 모델 요약

model_a.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']          

In [ ]:
image_path="/content/drive/MyDrive/L_TL2/cat/eye/normal"

#윈도우에서 시행시에
#image_path="C:/Users/VSA/Desktop/L_TL2/cat/eye/normal"

In [ ]:
#categories 는 유/무 구별

categories=["Y","N"] #0이 Y 1이 N

In [ ]:
#%% A모델 학습 각막궤양

X=[]
Y=[]

for idx, cate in enumerate(categories):
    label = [0 for _ in range(num_classes)]
    label[idx] = 1
    image_dir = image_path + '/a/'+cate+'/'

    for top, dir, files in os.walk(image_dir):
        for filename in files:
            if filename[-4:] != 'json':
                img = cv2.imread(image_dir + filename, 1) # 이미지를 BGR형식으로 호출
                img = cv2.resize(img, (200, 200)) #(400,400)이미지를 (200,200)이미지로 resize -> 크기의 배수로만 resize하여 화질이 뭉개지는 현상 방지
                # 이미지의 크기가 모델 ResNet 18 Layer모델에 입력 이미지 크기 (224,224)와 매치 되지 않아, zeropadding을 진행
                # 화질의 해상도를 최대한 유지 시키면서, 이미지의 크기를 키움.
                img = cv2.copyMakeBorder(img, 12, 12, 12, 12, cv2.BORDER_CONSTANT, value=0)
                X.append(img)
                Y.append(label)
# NUMPY형식으로 변환하고,
X=np.array(X)
X=X.astype(np.float32)/255.0 # 정규화 0~1의 값을 가질 수 있도록 만듦
Y=np.array(Y)
# 각각 라벨링 된 데이터에서, 10%를 제외한 나머지 90%를 학습 데이터에 활용 (데이터의 개수가 많지 않은 관계로 최대한 많은 데이터를 학습에 확보하기 위함.)
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.1)
# 학습 테이터, 테스트 데이터 개수 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
# 모델의 Learning rate, optimizer와 loss계산 규정
model_a.compile(optimizer=Adam(learning_rate=0.0003),loss='categorical_crossentropy',metrics=['accuracy'])
with tf.device('/GPU:0'):
  history=model_a.fit(x_train,y_train,epochs=100,batch_size=16)
# 테스트 데이터를 모델에 넣고, 이를 통해서, test accuracy,loss 반환
test_loss,test_acc=model_a.evaluate(x_test,y_test)
print("test accuracy:",test_acc)
# 데이터의 정확도를 기준으로한 그래프 작성
plt.plot(history.history['accuracy'])
plt.title('Train Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()
# 데이터의 loss를 기준으로 그래프 작성
plt.plot(history.history['loss'])
plt.title('TrainLoss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

(5625, 224, 224, 3)
(5625, 2)
(625, 224, 224, 3)
(625, 2)
Epoch 1/100
352/352 [==============================] - 110s 193ms/step - loss: 0.6235 - accuracy: 0.7275
Epoch 2/100
352/352 [==============================] - 68s 193ms/step - loss: 0.4093 - accuracy: 0.8295
Epoch 3/100
352/352 [==============================] - 68s 194ms/step - loss: 0.3578 - accuracy: 0.8521
Epoch 4/100
352/352 [==============================] - 68s 194ms/step - loss: 0.3364 - accuracy: 0.8656
Epoch 5/100
352/352 [==============================] - 68s 193ms/step - loss: 0.3239 - accuracy: 0.8692
Epoch 6/100
352/352 [==============================] - 68s 194ms/step - loss: 0.3200 - accuracy: 0.8756
Epoch 7/100
352/352 [==============================] - 68s 193ms/step - loss: 0.2934 - accuracy: 0.8843
Epoch 8/100
352/352 [==============================] - 68s 193ms/step - loss: 0.2785 - accuracy: 0.8935
Epoch 9/100
352/352 [==============================] - 68s 194ms/step - loss: 0.2895 - accuracy: 0.8862
Epoch

In [ ]:
# 모델 및 가중치 저장
model_a.save("/content/drive/MyDrive/model_a.h5")
model_a.save_weights("/content/drive/MyDrive/model_a_weight.h5")